In [23]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


from sklearn.metrics.pairwise import cosine_similarity

./RecommenderSystem.ipynb
./.git/config
./.git/HEAD
./.git/description
./.git/index
./.git/packed-refs
./.git/COMMIT_EDITMSG
./.git/FETCH_HEAD
./.git/objects/04/dddb75984d0187623acd34c617572da8796ab8
./.git/objects/ae/3c19dc95c27ab36fce9fb4bf0c9abd71a1c603
./.git/objects/f4/b9e637e8e0cfe5d90115dc14e95b1c0d226fb3
./.git/objects/e3/a6820dcb30d433634a81b1f1a9284a2292afe7
./.git/objects/c6/b7831384361daec0972507ef5f38c9bb18223c
./.git/objects/pack/pack-ec4e5e329ba013a02c0664c02a9f89cadf33ad57.pack
./.git/objects/pack/pack-ec4e5e329ba013a02c0664c02a9f89cadf33ad57.idx
./.git/objects/2a/23ce30bbb55de031e1b33a5bb1d93a0ecd7d80
./.git/objects/9a/0f6f5432f2f9d8564c3559fa2fe71f5663e1b1
./.git/objects/3f/5f0b61c3d037ec3f02a74f24017c7b038eb0dc
./.git/objects/d5/07e9338fa83ce86d1a438914a8f0a8f98dddc7
./.git/info/exclude
./.git/logs/HEAD
./.git/logs/refs/heads/RecommenderModel
./.git/logs/refs/heads/main
./.git/logs/refs/remotes/origin/HEAD
./.git/logs/refs/remotes/origin/RecommenderModel
./.git/hooks

# Load Data

In [24]:
books = pd.read_csv('/Users/thabochesane/Desktop/path2/books_data.csv')
reviews = pd.read_csv('/Users/thabochesane/Desktop/path2/Books_rating.csv')





In [ ]:
books.shape, reviews.shape

# Filtering

In [ ]:
reviews = reviews[['Id','User_id','review/score','Title']]

In [ ]:
reviews.rename(columns={'Id' : 'BookId', 'User_id' : 'UserId', 'review/score' : 'Review' , 'Title' : 'BookTitle' }, inplace= True)

In [ ]:
reviews.head(100)

# Get number of reviews per book

In [ ]:
numberofReviews = reviews.groupby('BookTitle').count()['Review'].reset_index()
numberofReviews.rename(columns={'Reviews' : 'NumberOfReviews'}, inplace= True)

In [ ]:
numberofReviews.head()

# Filter the users and books.

In [ ]:
userId = reviews.groupby('UserId').count()['Review'] > 200 
activeUsers = userId[userId].index
cleanedReviews = reviews[reviews['UserId'].isin(activeUsers)]
cleanedReviews

In [ ]:
x = cleanedReviews.groupby('BookTitle').count()['Review'] >= 50
popularBooks = x[x].index

reviewsDataFrame = cleanedReviews[cleanedReviews['BookTitle'].isin(popularBooks)]
reviewsDataFrame

In [ ]:
reviewsDataFrame.to_csv('/Users/thabochesane/Desktop/path2/reviews.csv')

In [ ]:
reviewsDataFrame.head()

# Check for Null values

In [ ]:
nullColumns = reviewsDataFrame.isnull().any()
print(nullColumns)

# Check for Not a number (NaN) values

In [ ]:
x = reviewsDataFrame.pivot_table(index='UserId', columns='BookTitle', values= 'Review')
x

In [ ]:
nan_counts = x.isna().sum()
print(nan_counts)

In [ ]:
x.fillna(0, inplace=True)
x

# simlarity analysis